In [2]:
import pandas as pd
import numpy as np

In [3]:
cdmx = pd.read_csv('listings - CDMX_clean.csv')
bang = pd.read_csv('listings_bangkok_clean.csv')
flor = pd.read_csv('listings_Florencia_clean.csv')
ny = pd.read_csv('listings - New York_clean.csv')

In [26]:
# Convertimos host_response_rate a float
# Primero quitamos el símbolo % y luego intentamos convertir a float
# Si hay valores que no se pueden convertir, los filtramos

# Para CDMX
cdmx['host_response_rate'] = cdmx['host_response_rate'].str.replace('%', '', regex=False)
cdmx = cdmx[cdmx['host_response_rate'].str.isnumeric().fillna(False)]
cdmx['host_response_rate'] = cdmx['host_response_rate'].astype(float) / 100

# Para Bangkok
bang['host_response_rate'] = bang['host_response_rate'].str.replace('%', '', regex=False)
bang = bang[bang['host_response_rate'].str.isnumeric().fillna(False)]
bang['host_response_rate'] = bang['host_response_rate'].astype(float) / 100

# Para Florencia
flor['host_response_rate'] = flor['host_response_rate'].str.replace('%', '', regex=False)
flor = flor[flor['host_response_rate'].str.isnumeric().fillna(False)]
flor['host_response_rate'] = flor['host_response_rate'].astype(float) / 100

# Para Nueva York
ny['host_response_rate'] = ny['host_response_rate'].str.replace('%', '', regex=False)
ny = ny[ny['host_response_rate'].str.isnumeric().fillna(False)]
ny['host_response_rate'] = ny['host_response_rate'].astype(float) / 100

C:\Users\angel\AppData\Local\Temp\ipykernel_13696\2179240890.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bang = bang[bang['host_response_rate'].str.isnumeric().fillna(False)]


In [34]:
# Convertimos host_acceptance_rate a float
# Primero quitamos el símbolo % y luego intentamos convertir a float
# Si hay valores que no se pueden convertir, los filtramos

# Para CDMX
cdmx['host_acceptance_rate'] = cdmx['host_acceptance_rate'].str.replace('%', '', regex=False)
cdmx = cdmx[cdmx['host_acceptance_rate'].str.isnumeric().fillna(False)]
cdmx['host_acceptance_rate'] = cdmx['host_acceptance_rate'].astype(float) / 100

# Para Bangkok
bang['host_acceptance_rate'] = bang['host_acceptance_rate'].str.replace('%', '', regex=False)
bang = bang[bang['host_acceptance_rate'].str.isnumeric().fillna(False)]
bang['host_acceptance_rate'] = bang['host_acceptance_rate'].astype(float) / 100

# Para Florencia
flor['host_acceptance_rate'] = flor['host_acceptance_rate'].str.replace('%', '', regex=False)
flor = flor[flor['host_acceptance_rate'].str.isnumeric().fillna(False)]
flor['host_acceptance_rate'] = flor['host_acceptance_rate'].astype(float) / 100

# Para Nueva York
ny['host_acceptance_rate'] = ny['host_acceptance_rate'].str.replace('%', '', regex=False)
ny = ny[ny['host_acceptance_rate'].str.isnumeric().fillna(False)]
ny['host_acceptance_rate'] = ny['host_acceptance_rate'].astype(float) / 100

C:\Users\angel\AppData\Local\Temp\ipykernel_13696\1061365228.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bang = bang[bang['host_acceptance_rate'].str.isnumeric().fillna(False)]


In [6]:
# Quitaremos los outliers de las columnas 'price' de bangkok, las cuales ya están en float
# Usaremos el método IQR para detectar outliers

def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Aplicamos la función a las columnas 'price' de cada bangkok
bang = remove_outliers_iqr(bang, 'price')

In [7]:
bang['price'].max(), bang['price'].min()

(3529.88, 40.02)

In [19]:
# Quitamos todos los nulos de cdmx de la columna room_type
cdmx = cdmx[~cdmx['room_type'].isnull()]

In [12]:
# Convertimos a float las columnas de precios de bangkok
bang['price'] = bang['price'].str.replace('$', '').str.replace(',', '').astype(float)
bang['price'].dtypes

dtype('float64')

In [23]:
# Rellenamos los nulos de la columna price de bangkok con interpolación
bang['price'] = bang['price'].interpolate(method='linear', limit_direction='both')

In [25]:
# Vamos a pasar el price a mxn para que la comparación sea justa

# Definimos los tipos de cambio (a MXN)
usd_to_mxn = 19.35  # Tipo de cambio USD a MXN
eur_to_mxn = 21.7  # Tipo de cambio EUR a MXN
thb_to_mxn = 0.58   # Tipo de cambio THB (Baht tailandés) a MXN

# CDMX ya está en MXN, no necesita conversión
# cdmx['price_mxn'] = cdmx['price']

# Bangkok está en THB (Baht tailandés)
# Nos aseguramos que la columna price sea numérica
bang['price'] = bang['price'] * thb_to_mxn

# Florencia está en EUR
# Nos aseguramos que la columna price sea numérica
flor['price'] = flor['price'] * eur_to_mxn

# Nueva York está en USD
# Nos aseguramos que la columna price sea numérica
ny['price'] = ny['price'] * usd_to_mxn

In [33]:
# Quitamos los nulos en la columna review_scores_rating de bangkok usando interpolación
bang['review_scores_rating'].interpolate(method='linear', limit_direction='both', inplace=True)

C:\Users\angel\AppData\Local\Temp\ipykernel_14676\306439828.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  bang['review_scores_rating'].interpolate(method='linear', limit_direction='both', inplace=True)


In [8]:
# Exportamos los dataframes a csv
bang.to_csv('listings_bangkok_clean.csv', index=False)
cdmx.to_csv('listings - CDMX_clean.csv', index=False)
flor.to_csv('listings_Florencia_clean.csv', index=False)
ny.to_csv('listings - New York_clean.csv', index=False)